**Importing librosa library**

In [15]:
import librosa
import os
import matplotlib.pyplot as plt
import librosa.display
import numpy as np

import pandas as pd
from tqdm.notebook import tqdm


**Loading the audio files**

In [18]:
# Load an audio file

# Get the current working directory
current_directory = os.getcwd()

# Specify the path to the audio file relative to the current directory
#audio_file_path = os.path.join(current_directory, 'Nicole-Documents', 'KAGGLE', 'AUDIO')
audio_file_path = os.path.join(current_directory, 'Nicole-Documents', 'DEMONSTRATION', 'DEMONSTRATION')
audio_file_path2 = os.path.join(current_directory, 'Nicole-Documents', 'KAGGLE', 'AUDIO','REAL','biden-original.wav')
y, sr = librosa.load(audio_file_path2)

# Display the waveform
#plt.figure(figsize=(12, 4))
#librosa.display.waveshow(y, sr=sr)
#plt.title('Waveform')
#plt.show()

In [3]:
print(len(y))

13230066


In [4]:
y

array([ 0.02332861,  0.03871367,  0.02713641, ...,  0.02201173,
        0.00891917, -0.00109065], dtype=float32)

In [5]:
sr

22050

In [6]:
chroma = librosa.feature.chroma_cqt(y=y,sr=sr)

In [7]:
chroma

array([[0.37303543, 0.49351314, 0.36058465, ..., 1.        , 1.        ,
        0.7891258 ],
       [0.46651   , 0.56956625, 0.38559422, ..., 0.48033077, 0.36449018,
        0.38136163],
       [0.42050812, 0.5437503 , 0.5659621 , ..., 0.12449706, 0.18183646,
        0.24776603],
       ...,
       [0.45227924, 0.5147358 , 0.38905585, ..., 0.2507778 , 0.529811  ,
        0.8691794 ],
       [0.37522292, 0.41887188, 0.29968226, ..., 0.4005598 , 0.6486622 ,
        0.8880082 ],
       [0.3451065 , 0.43786275, 0.28772122, ..., 0.5604363 , 0.9982105 ,
        1.        ]], dtype=float32)

In [8]:
len(chroma)

12

In [9]:
len(chroma[2])

25840

In [10]:
!pip install pysound

In [22]:
###
"""
THIS CODE GENERATES A MODIFIED VERSION OF THE ORIGINAL CSV DATASET THAT
INCLUDES A COLUMN FOR 'original_sample' WHICH LISTS THE AUDIO SAMPLE
THAT THE 1 SECOND SEGMENT ORIGINATES FROM
"""
###

# Function to extract features from an audio file
def extract_features(file_path, segment_length, file_name):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path)
        # Calculate the number of segments based on the segment length and audio length
        num_segments = int(np.ceil(len(y) / float(segment_length * sr)))

        # Initialize a list to store the features for this file
        features = []

        # Extract features for each segment
        for i in range(num_segments):
            # Calculate start and end frame for the current segment
            start_frame = i * segment_length * sr
            end_frame = min(len(y), (i + 1) * segment_length * sr)
            
            # Extract audio for this segment
            y_segment = y[start_frame:end_frame]

            # Extract features
            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr,n_fft=2048, hop_length=512))
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr)
            mfccs_mean = np.mean(mfccs, axis=1)
            
            # Append the extracted features to the list
            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean, file_name])

        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Function to create the dataset
def create_dataset(audio_dir, segment_length):
    labels = ['NEW_FAKE']
    feature_list = []

    # Iterate over all files in the audio_dir
    for label in labels:
        print(f'Processing {label} files...')
        files = os.listdir(os.path.join(audio_dir, label))
        # Wrap the files iterable with tqdm to show the progress bar
        for file in tqdm(files, desc=f"{label} Files Progress"):
            file_path = os.path.join(audio_dir, label, file)
            # Extract the file name without the extension
            file_name = os.path.splitext(file)[0]
            # Extract features for the current file
            file_features = extract_features(file_path, segment_length, file_name)
            if file_features:
                # Append features of all segments along with the label to the dataset
                for segment_features in file_features:
                    feature_list.append(segment_features + [label])
                    
    # Create a DataFrame with the dataset
    df = pd.DataFrame(feature_list, columns=['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'origin_sample', 'LABEL'])
    
    return df

# Directory where the 'FAKE' and 'REAL' folders are located
audio_dir = audio_file_path

# Length of the audio segments in seconds
segment_length = 1  # for example, 5 seconds

# Create the dataset
dataset = create_dataset(audio_dir, segment_length)

# Save the dataset to a CSV file
#csv_output_path = 
#dataset.to_csv(csv_output_path, index=False)

#print(f'Dataset created and saved to {csv_output_path}')

Processing NEW_FAKE files...


NEW_FAKE Files Progress:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
len(y)/2

6615033.0

In [23]:
csv_output_path = 'audio_features_biden_AI.csv'
dataset.to_csv(csv_output_path, index=False)

print(f'Dataset created and saved to {csv_output_path}')

Dataset created and saved to audio_features_biden_AI.csv


In [24]:
# Count the occurrences of each label
label_counts = dataset['LABEL'].value_counts()

# Print the counts
print(label_counts)

LABEL
NEW_FAKE    80
Name: count, dtype: int64
